# Ejemplo de regresión RN

## Datos

In [1]:
# DATOS
import pandas as pd
import numpy as np
datos = pd.read_csv('Advertising.csv')

In [2]:
datos

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


## Train and test

In [3]:
from sklearn.model_selection import train_test_split

X = datos.iloc[:, :3]
y = datos.iloc[:,-1]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.75 ,random_state=0)

## Modelo

In [4]:
# Importamos la función para REGRESIÓN
from sklearn.neural_network import MLPRegressor

In [5]:
# Creamos un modelo con una sola capa
modelo_1 = MLPRegressor(hidden_layer_sizes=(5),  # una capa con 5 neuronas
                        solver = 'lbfgs',        # algoritmo de optimización, para sets pequeños este es mejor
                        max_iter = 1000,         # num max de iteraciones de entrenamiento
                        random_state = 123)      # semilla de aleatoriedad

In [6]:
# Ajustamos el modelo a los datos de entrenamiento
modelo_1.fit(Xtrain, ytrain)

MLPRegressor(hidden_layer_sizes=5, max_iter=1000, random_state=123,
             solver='lbfgs')

## Evaluación del primer modelo

In [7]:
# Realizamos predicciones usando los datos de prueba
y_pred = modelo_1.predict(Xtest)

In [8]:
# Error de test del modelo inicial
#-------------------------------------------------------------------------------
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(ytest, y_pred, squared = False)

print(f"El error cuadrático medio (rmse) de test es: {rmse}")
print(f"Las predicciones del modelo se alejan en promedio {rmse} unidades del valor real.")

El error cuadrático medio (rmse) de test es: 2.003122984156475
Las predicciones del modelo se alejan en promedio 2.003122984156475 unidades del valor real.


## Optimización de hiperparámetros

In [9]:
from sklearn.model_selection import RandomizedSearchCV

 # Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_distributions = {'hidden_layer_sizes': [(10), (20), (10, 10)],
                       'alpha': np.logspace(-3, 3, 10),
                       'learning_rate_init': [0.001, 0.01]
                      }

In [10]:
# Búsqueda por validación cruzada
# ==============================================================================
grid = RandomizedSearchCV(estimator = MLPRegressor(solver = 'lbfgs', max_iter= 10000),
                          param_distributions = param_distributions,
                          n_iter = 50,
                          scoring = 'neg_mean_squared_error',
                          cv = 5,
                          random_state = 123)

grid.fit(X = Xtrain, y = ytrain);

In [11]:
# Los mejores parámetros
grid.best_params_

{'learning_rate_init': 0.01, 'hidden_layer_sizes': 20, 'alpha': 10.0}

In [12]:
# El modelo con los mejores parámetros está almacenado en:
modelo_final = grid.best_estimator_
modelo_final

MLPRegressor(alpha=10.0, hidden_layer_sizes=20, learning_rate_init=0.01,
             max_iter=10000, solver='lbfgs')

## Evaluación del modelo con los mejores hiperparámetros

In [13]:
# Realizamos predicciones usando los datos de prueba
y_pred2 = modelo_final.predict(Xtest)

In [14]:
# Error de test del modelo inicial
#-------------------------------------------------------------------------------
from sklearn.metrics import mean_squared_error

rmse2 = mean_squared_error(ytest, y_pred2, squared = False)

print(f"El error cuadrático medio (rmse) de test es: {rmse2}")
print(f"Las predicciones del modelo final se alejan en promedio {rmse2} unidades del valor real.")

El error cuadrático medio (rmse) de test es: 0.7231669792060338
Las predicciones del modelo final se alejan en promedio 0.7231669792060338 unidades del valor real.


## Regresión

In [15]:
# Modelo de regresión múltiple
import statsmodels.api as sm
X_train = sm.add_constant(Xtrain, prepend=True)
modelo_reg = sm.OLS(endog=ytrain, exog=X_train).fit()

# Predicciones
X_test = sm.add_constant(Xtest, prepend=True)
predicciones = modelo_reg.predict(X_test)

# Error para comparar con el de la red neuronal
mean_squared_error(ytest, predicciones, squared = False)

D:\Documentos\Programas\Miniconda3\envs\Entorno01\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


2.0031219440955446

Notemos que la red neuronal nos ofrece muchas mejores predicciones que una regresión.